In [ ]:
import requests
import pandas as pd


In [ ]:

# Base URL for the tournament's hole stats (replace with your specific tournament ID and round ID)
base_url = "https://www.pdga.com/api/v1/feat/stats/hole-stats-breakdown/122324129/"

# Number of holes in the course
num_holes = 18

# List to store data for all holes
all_holes_stats = []

for hole_num in range(1, num_holes + 1):
    # Construct the full URL for the current hole
    url = f"{base_url}{hole_num}"
    
    # Fetch the data
    response = requests.get(url)
    data = response.json()
    
    # Extract and store stats for each player in the current hole
    for entry in data:
        player_info = entry['liveScore']['liveResult']
        breakdown = entry['breakdown']
        
        # Combine player info with breakdown and add hole number
        player_stats = {
            'Hole': hole_num,
            'FirstName': player_info['firstName'],
            'LastName': player_info['lastName'],
            'PDGANum': player_info['pdgaNum'],
            'Place': player_info['place'],
            'TotalScore': player_info['total'],
            'ToPar': player_info['toPar'],
            'Driving': breakdown.get('driving', None),
            'Scramble': breakdown.get('scramble', None),
            'Green': breakdown.get('green', None),
            'C1x': breakdown.get('c1x', 0),
            'C1': breakdown.get('c1', 0),
            'C2': breakdown.get('c2', 0),
            'ThrowIn': breakdown.get('throwIn', 0),
            'OB': breakdown.get('ob', 0),
            'Hazard': breakdown.get('hazard', 0),
            'MissedMando': breakdown.get('missedMando', 0),
            'LostDisc': breakdown.get('lostDisc', 0),
            'Penalty': breakdown.get('penalty', 0),
            'HoleScore': breakdown.get('holeScore', None)
        }
        all_holes_stats.append(player_stats)

In [ ]:

# Convert the list to a DataFrame
df = pd.DataFrame(all_holes_stats)

# Display the data
print(df)


In [ ]:

# Save to CSV for further analysis
df.to_csv('tournament_hole_stats.csv', index=False)


In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
# URL of the PDGA page listing the tournaments (tournament search page sorted for ES and M tournaments within the date range of this year)
tournaments_url = 'https://www.pdga.com/tour/search?OfficialName=&td=&date_filter%5Bmin%5D%5Bdate%5D=2024-02-01&date_filter%5Bmax%5D%5Bdate%5D=2025-08-14&Tier%5B%5D=ES&Tier%5B%5D=M' 


In [ ]:

# Send a GET request to the page
response = requests.get(tournaments_url)
soup = BeautifulSoup(response.text, 'html.parser')


In [ ]:


# Initialize a dictionary to store tournament names and IDs
tournaments_dict = {}

# Find the table containing the events
table = soup.find('div', class_='table-container')


In [ ]:

# If the table exists, proceed
if table:
    # Find all rows in the table body
    rows = table.find('tbody').find_all('tr')

    for row in rows:
        # Find the cell with the tournament name
        tournament_cell = row.find('td', class_='views-field-OfficialName')
        
        if tournament_cell:
            # Extract the link and name
            link = tournament_cell.find('a')
            if link:
                tournament_name = link.text.strip()
                href = link['href']
                tournament_id = href.split('/')[-1]
                
                # Add to dictionary
                tournaments_dict[tournament_name] = tournament_id


In [ ]:

# Print the resulting dictionary
print(tournaments_dict)
# use this dictionary in asyncio_run.py to find round ids

In [ ]:
# round_ids captured by asyncio_run.py 
round_ids = {'78584': {1: '122299849'}, '77775': {1: '122287100', 2: '122287102', 3: '122287104'}, '77758': {1: '122301092', 2: '122301094', 3: '122301096', 4: '122314192'}, '77759': {1: '122288247', 2: '122288249', 3: '122288251'}, '77091': {}, '77760': {1: '122291884', 2: '122291886', 3: '122291888'}, '77761': {1: '122286664', 2: '122286666', 3: '122286668'}, '77762': {1: '122286804', 2: '122286806', 3: '122286808'}, '77099': {1: '122283440', 2: '122283442', 3: '122283444', 4: '122360073'}, '77763': {1: '122287578', 2: '122287580', 3: '122287582'}, '78193': {1: '122292625', 2: '122292627', 3: '122292629'}, '77764': {1: '122286793', 2: '122286794', 3: '122286795'}, '77765': {1: '122299697', 2: '122299699', 3: '122299701', 4: '122299703'}, '77766': {1: '122316840', 2: '122316842', 3: '122316844'}, '77093': {}, '77133': {}, '77094': {}, '78194': {1: '122292631', 2: '122292633', 3: '122292635'}, '78271': {1: '122312714', 2: '122312716', 3: '122312718'}, '78195': {1: '122292637', 2: '122292639', 3: '122292641'}, '77768': {1: '122301365', 2: '122301367', 3: '122301369'}, '78196': {1: '122292643', 2: '122292645', 3: '122292647'}, '77095': {}, '77750': {1: '122309308', 2: '122309309', 3: '122309310', 4: '122309312'}, '78197': {1: '122292649', 2: '122292651', 3: '122292653'}}

In [ ]:
tournament_rounds = {}
for tournament_name, tournament_id in tournaments_dict.items():
    for tid, rounds in round_ids.items():
        if tournament_id == tid:
            tournament_rounds[tournament_name] = round_ids[tournament_id]

print(tournament_rounds)

In [ ]:
# Function to pull stats from each round with an associated round id
# Not ready yet 
# def get_tournament_stats(tournament_ids):
#     # Base URL for the tournament's hole stats (replace with your specific tournament ID and round ID)
#     base_url = "https://www.pdga.com/api/v1/feat/stats/hole-stats-breakdown/"

#     # Number of holes in the course
#     num_holes = 18

#     # List to store data for all holes
#     all_holes_stats = []
#     for 
#         for hole_num in range(1, num_holes + 1):
#             # Construct the full URL for the current hole
#             url = f"{base_url}{hole_num}"
            
#             # Fetch the data
#             response = requests.get(url)
#             data = response.json()
            
#             # Extract and store stats for each player in the current hole
#             for entry in data:
#                 player_info = entry['liveScore']['liveResult']
#                 breakdown = entry['breakdown']
                
#                 # Combine player info with breakdown and add hole number
#                 player_stats = {
#                     'Hole': hole_num,
#                     'FirstName': player_info['firstName'],
#                     'LastName': player_info['lastName'],
#                     'PDGANum': player_info['pdgaNum'],
#                     'Place': player_info['place'],
#                     'TotalScore': player_info['total'],
#                     'ToPar': player_info['toPar'],
#                     'Driving': breakdown.get('driving', None),
#                     'Scramble': breakdown.get('scramble', None),
#                     'Green': breakdown.get('green', None),
#                     'C1x': breakdown.get('c1x', 0),
#                     'C1': breakdown.get('c1', 0),
#                     'C2': breakdown.get('c2', 0),
#                     'ThrowIn': breakdown.get('throwIn', 0),
#                     'OB': breakdown.get('ob', 0),
#                     'Hazard': breakdown.get('hazard', 0),
#                     'MissedMando': breakdown.get('missedMando', 0),
#                     'LostDisc': breakdown.get('lostDisc', 0),
#                     'Penalty': breakdown.get('penalty', 0),
#                     'HoleScore': breakdown.get('holeScore', None)
#                 }
#                 all_holes_stats.append(player_stats)

In [ ]:

# Function to calculate summary stats by round

def summary_stats(round_df):
    # Initialize summary statistics dictionary
    summary_stats = {}

    # Group by player
    players = round_df.groupby(['FirstName', 'LastName', 'PDGANum'])

    # Process each player
    for player, data in players:
        c1x_makes = data[(data['ThrowIn'] >= 11) & (data['ThrowIn'] <= 33)].shape[0]
        c1x_attempts = data['C1x'].sum() + data['C1'].sum() - data[data['ThrowIn'] < 11].shape[0]
        
        c2_makes = data[(data['ThrowIn'] >= 34) & (data['ThrowIn'] <= 66)].shape[0]
        c2_attempts = data['C2'].sum()
        
        parked_holes = data[data['Green'] == 'parked'].shape[0]
        c1_driving_holes = data[(data['Green'] == 'parked') | (data['Green'] == 'c1')].shape[0]
        c2_driving_holes = data[(data['Green'] == 'parked') | (data['Green'] == 'c1') | (data['Green'] == 'c2')].shape[0]
        
        total_holes = data.shape[0]
        
        average_throw_in_distance = data[data['ThrowIn'] > 0]['ThrowIn'].mean()
        
        summary_stats[player] = {
            'C1x Putting %': (c1x_makes / c1x_attempts * 100) if c1x_attempts > 0 else None,
            'C2 Putting %': (c2_makes / c2_attempts * 100) if c2_attempts > 0 else None,
            'Parked %': (parked_holes / total_holes * 100) if total_holes > 0 else None,
            'C1 Driving %': (c1_driving_holes / total_holes * 100) if total_holes > 0 else None,
            'C2 Driving %': (c2_driving_holes / total_holes * 100) if total_holes > 0 else None,
            'Average Throw-In Distance': average_throw_in_distance
        }

    # Convert to DataFrame
    summary_df = pd.DataFrame.from_dict(summary_stats, orient='index')

    return summary_df


In [ ]:
# Test with a round

# Base URL for the tournament's hole stats (replace with your specific tournament ID and round ID)
base_url = "https://www.pdga.com/api/v1/feat/stats/hole-stats-breakdown/122324129/"

# Number of holes in the course
num_holes = 18

# List to store data for all holes
all_holes_stats = []

for hole_num in range(1, num_holes + 1):
    # Construct the full URL for the current hole
    url = f"{base_url}{hole_num}"
    
    # Fetch the data
    response = requests.get(url)
    data = response.json()
    
    # Extract and store stats for each player in the current hole
    for entry in data:
        player_info = entry['liveScore']['liveResult']
        breakdown = entry['breakdown']
        
        # Combine player info with breakdown and add hole number
        player_stats = {
            'Hole': hole_num,
            'FirstName': player_info['firstName'],
            'LastName': player_info['lastName'],
            'PDGANum': player_info['pdgaNum'],
            'Place': player_info['place'],
            'TotalScore': player_info['total'],
            'ToPar': player_info['toPar'],
            'Driving': breakdown.get('driving', None),
            'Scramble': breakdown.get('scramble', None),
            'Green': breakdown.get('green', None),
            'C1x': breakdown.get('c1x', 0),
            'C1': breakdown.get('c1', 0),
            'C2': breakdown.get('c2', 0),
            'ThrowIn': breakdown.get('throwIn', 0),
            'OB': breakdown.get('ob', 0),
            'Hazard': breakdown.get('hazard', 0),
            'MissedMando': breakdown.get('missedMando', 0),
            'LostDisc': breakdown.get('lostDisc', 0),
            'Penalty': breakdown.get('penalty', 0),
            'HoleScore': breakdown.get('holeScore', None)
        }
        all_holes_stats.append(player_stats)

In [ ]:
# Convert the list to a DataFrame
df = pd.DataFrame(all_holes_stats)

# Display the data
print(df)

In [ ]:
test_df = summary_stats(df)

In [ ]:
print(test_df)